In [1]:
from vertexai.preview.language_models import ChatModel

class PaLMWrapper:
    def __init__(self):
        self.chat_model = ChatModel.from_pretrained("chat-bison@001")
        self.parameters = {
            "temperature": 0.2,
            "max_output_tokens": 256,
            "top_p": 0.95,
        }
    
    def generate_response(self, context, message):
        chat = self.chat_model.start_chat(context=context)
        response = chat.send_message(message, **self.parameters)
        return response.text

2023-12-01 15:27:55.698272: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-01 15:28:00.130803: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-12-01 15:28:00.132612: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [2]:
pw = PaLMWrapper()
response = pw.generate_response(context='you are a london weather forecaster', message='whats the weather like today')
response

'The weather today in London is sunny with a high of 15 degrees Celsius.'

In [3]:
import pandas as pd

import vertexai
from vertexai.preview.language_models import TextEmbeddingModel
from google.cloud import aiplatform

class VertexAIVectorStore:
    def __init__(self):
        
        # god awful way of doing it, should be a config and passed through but oh well hacky hack
        self.gen_ai_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(
            index_endpoint_name="projects/playpen-8d8611/locations/europe-west2/indexEndpoints/5891412000042385408"
        )

        self.gen_ai_index = aiplatform.MatchingEngineIndex(
            index_name="projects/playpen-8d8611/locations/europe-west2/indexes/2680908415680643072"
        )
        
        self.model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")
        self.df = pd.read_csv('text_data_g_embedding.csv')
        
    def search(self, input, k=3):
        embedding_vec =  self.model.get_embeddings([input])[0].values #Send request to embedding model to generate the embedding vector

        #find neighbours using vector search
        neighbours = self.gen_ai_index_endpoint.find_neighbors(
            deployed_index_id="gen_ai_deployed_index",
            queries=[embedding_vec],
            num_neighbors=k,
        )[0]
        
        results = []
        for nb in neighbours:
            text_id = nb.id
            text = self.df.iloc[int(nb.id)]['text']
            score = nb.distance
            results.append((text_id, text, score))
        
        return results


In [4]:
vai = VertexAIVectorStore()
vector = vai.search(input='what is credit card fraud')
vector

[('41',
  'Security & fraudLost or stolen cardsReport a lost or stolen Business CardReport a lost or stolen Business payment cardReport a lost or stolen Corporate CardReport a lost or stolen Corpoarate cardReport a fraudReport a fraud on our business accountsMake a complaintReport fraudulent use of online bankingStaying safeProtect your business from fraudManage the cyber threat to your business',
  0.7093294858932495),
 ('55',
  "Report\xa0fraud on online business bankingCall us immediately\xa0 if you think any of your Lloyds Business Bank accounts have been:accessed by a third party\xa0ortargeted for\xa0fraud.Please get in touch even if you haven't suffered any loss from the attempted fraud.This page is for business customersFind out how to report fraud if you’re a personal customerReport FraudBy PhoneOnline for Business\xa00800 056 3099Commercial Banking Online\xa00800 169 1296+44 1293 762 380 from abroad.\xa0Lines are open 24 hours.You'll need to tell us the reason for your call so

In [19]:
# custom RAG anyone?

class RAG:
    def __init__(self, vector_store, palm_wrapper):
        self.vector_store = vector_store
        self.palm_wrapper = palm_wrapper
        # at the request of mathew for memory
        self.conversation_history = []
    
    def generate(self, query, k=3):
        self.conversation_history.append(query)
        
        search_results = self.vector_store.search(query, k)
        contexts = [result[1] for result in search_results]
        
        history_context = [resp for resp in self.conversation_history[-k:]]
        combined_context = '\n'.join(history_context + contexts)
        response = self.palm_wrapper.generate_response(combined_context[:20000], query)
        
        self.conversation_history.append((query, response))
        
        sources = [(result[0]) for result in search_results]
        
        return response, sources

In [20]:
rag = RAG(vai, pw)
query = "how do i change my address?"
response, sources = rag.generate(query)
print(response)
print('sources used:', sources)

To change your business address, you can update it online or complete a form and send it to us.

To update your address online, you must be a full access user and have your card reader ready to approve your changes.

In Online for Business, log in and select Admin.

In the Business Details section, select Your addresses.

This screen displays your address details. Find the address you want to update and select Change.

Follow the onscreen guidance.

You'll need to verify yourself using your card and reader once you've made your changes. You'll then get a reference number.

If you're not a full access user or don't have your card reader ready, you can complete a form and send it to us.

To get you to the right form, we just need a bit of information from you.

Do you have a named Relationship Manager?

Select Yes or No.

Is your turnover over £25 million?

Select Yes or No.
sources used: ['24', '26', '45']
